[View in Colaboratory](https://colab.research.google.com/github/iamrajee/Data-Analysis/blob/master/dragnn.ipynb)

In [4]:
!apt-get install -y graphviz libgraphviz-dev libopenblas-base libpng16-16 libxft2 python-pip python-mock
!pip install pygraphviz --install-option="--include-path=/usr/include/graphviz" --install-option="--library-path=/usr/lib/graphviz/"
!pip install 'ipython<6.0' protobuf numpy scipy jupyter syntaxnet-with-tensorflow
!python -m jupyter_core.command nbextension enable --py --sys-prefix widgetsnbextension
!python -c 'import dragnn.python.load_dragnn_cc_impl, syntaxnet.load_parser_ops'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
graphviz is already the newest version (2.38.0-16ubuntu2).
libgraphviz-dev is already the newest version (2.38.0-16ubuntu2).
libpng16-16 is already the newest version (1.6.34-1).
libxft2 is already the newest version (2.3.2-1).
python-mock is already the newest version (2.0.0-3).
libopenblas-base is already the newest version (0.2.20+ds-4).
python-pip is already the newest version (9.0.1-2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
/usr/local/lib/python2.7/dist-packages/pip/_internal/commands/install.py:199: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [5]:
!apt-get install swig
!pip install -U protobuf==3.4.0
!pip install -U autograd==1.1.13
!pip install mock
!pip install asciitree
!pip install numpy
!apt-get install -y graphviz libgraphviz-dev
!pip install pygraphviz --install-option="--include-path=/usr/include/graphviz" --install-option="--library-path=/usr/lib/graphviz/"
!apt-get install pkg-config zip g++ zlib1g-dev unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.10-1.2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Requirement already up-to-date: protobuf==3.4.0 in /usr/local/lib/python2.7/dist-packages (3.4.0)
Requirement not upgraded as not directly required: six>=1.9 in /usr/local/lib/python2.7/dist-packages (from protobuf==3.4.0) (1.11.0)
Requirement not upgraded as not directly required: setuptools in /usr/local/lib/python2.7/dist-packages (from protobuf==3.4.0) (39.2.0)
Requirement already up-to-date: autograd==1.1.13 in /usr/local/lib/python2.7/dist-packages (1.1.13)
Requirement not upgraded as not directly required: numpy>=1.12 in /usr/local/lib/python2.7/dist-packages (from autograd==1.1.13) (1.14.3)
Requirement not upgraded as not directly required: future>=0.15.2 in /usr/local/lib/python2.7/dist-packages (from autograd==1.1.13) (0.16.0)
Reading package lists... Done
Building depend

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os.path

import tensorflow as tf
from google.protobuf import text_format
from dragnn.protos import spec_pb2
from dragnn.python import graph_builder
from dragnn.python import lexicon
from dragnn.python import spec_builder
from dragnn.python import visualization
from syntaxnet import sentence_pb2

import dragnn.python.load_dragnn_cc_impl
import syntaxnet.load_parser_ops

In [7]:
!ls

datalab


In [8]:
from google.colab import files
uploaded = files.upload()

Saving sentence.prototext to sentence.prototext


In [0]:
f = open('sentence.prototext', 'r')
training_sentence = f.read()

In [10]:
training_sentence

'text: "fair enough; you people have eaten me."\ntoken {\n  word: "fair"\n  start: 0\n  end: 3\n  tag: "JJ"\n  category: "ADJ"\n  label: "ROOT"\n  break_level: NO_BREAK\n}\ntoken {\n  word: "enough"\n  start: 5\n  end: 10\n  head: 0\n  tag: "RB"\n  category: "ADV"\n  label: "advmod"\n  break_level: SPACE_BREAK\n}\ntoken {\n  word: ";"\n  start: 11\n  end: 11\n  head: 0\n  tag: ":"\n  category: "."\n  label: "p"\n  break_level: NO_BREAK\n}\ntoken {\n  word: "you"\n  start: 13\n  end: 15\n  head: 4\n  tag: "PRP"\n  category: "PRON"\n  label: "dep"\n  break_level: SPACE_BREAK\n}\ntoken {\n  word: "people"\n  start: 17\n  end: 22\n  head: 6\n  tag: "NNS"\n  category: "NOUN"\n  label: "nsubj"\n  break_level: SPACE_BREAK\n}\ntoken {\n  word: "have"\n  start: 24\n  end: 27\n  head: 6\n  tag: "VBP"\n  category: "VERB"\n  label: "aux"\n  break_level: SPACE_BREAK\n}\ntoken {\n  word: "eaten"\n  start: 29\n  end: 33\n  head: 0\n  tag: "VBN"\n  category: "VERB"\n  label: "parataxis"\n  break_level

In [0]:
lexicon_dir = '/tmp/tutorial/lexicon'
if not os.path.isdir(lexicon_dir):
  os.makedirs(lexicon_dir)

In [0]:
def main(argv):
  del argv  # unused
  # Constructs lexical resources for SyntaxNet in the given resource path, from
  # the training data.
  lexicon.build_lexicon(
      lexicon_dir,
      training_sentence,
      training_corpus_format='sentence-prototext')
  tagger = spec_builder.ComponentSpecBuilder('tagger')
  tagger.set_network_unit(name='FeedForwardNetwork', hidden_layer_sizes='256')
  tagger.set_transition_system(name='tagger')
  tagger.add_fixed_feature(name='words', fml='input.word', embedding_dim=64)
  tagger.add_rnn_link(embedding_dim=-1)
  tagger.fill_from_resources(lexicon_dir)
  
  
  # Construct the ComponentSpec for parsing.
  parser = spec_builder.ComponentSpecBuilder('parser')
  parser.set_network_unit(
      name='FeedForwardNetwork',
      hidden_layer_sizes='256',
      layer_norm_hidden='True')
  parser.set_transition_system(name='arc-standard')
  parser.add_token_link(
      source=tagger,
      fml='input.focus stack.focus stack(1).focus',
      embedding_dim=32,
      source_layer='logits')
  parser.add_link(
      source=parser,  # recurrent connection
      name='rnn-stack',  # unique identifier
      fml='stack.focus stack(1).focus',  # look for both stack tokens
      source_translator='shift-reduce-step',  # maps token indices -> step
      embedding_dim=32)  # project down to 32 dims
  parser.fill_from_resources(lexicon_dir)
  
  
  
  
  
  master_spec = spec_pb2.MasterSpec()
  master_spec.component.extend([tagger.spec, parser.spec])

  hyperparam_config = spec_pb2.GridPoint()
  
  
  
  
  
  
  # Build the TensorFlow graph.
  graph = tf.Graph()
  with graph.as_default():
    builder = graph_builder.MasterBuilder(master_spec, hyperparam_config)

    target = spec_pb2.TrainTarget()
    target.name = 'all'
    target.unroll_using_oracle.extend([True, True])
    dry_run = builder.add_training_from_config(target, trace_only=True)

    
    
    
    
  # Read in serialized protos from training data.
  sentence = sentence_pb2.Sentence()
  text_format.Merge(open(training_sentence).read(), sentence)
  training_set = [sentence.SerializeToString()]

  
  
  
  with tf.Session(graph=graph) as sess:
    # Make sure to re-initialize all underlying state.
    sess.run(tf.initialize_all_variables())
    traces = sess.run(
        dry_run['traces'], feed_dict={dry_run['input_batch']: training_set})
    
    
    
    
  with open('dragnn_tutorial_2.html', 'w') as f:
    f.write(
        visualization.trace_html(
            traces[0], height='400px', master_spec=master_spec).encode('utf-8'))


In [0]:
if __name__ == '__main__':
  tf.app.run()